In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark as ps
import scipy.stats as stats
import sys
sys.path.append("..")
from src.support_functions import get_covid_data, fixing_datetime, get_zip_income, clean_up_turnstile_data

plt.style.use('ggplot')


In [2]:
data_by_zip, tests_by_day, tests_by_boro = get_covid_data()

In [3]:
data_by_zip = data_by_zip.rename(columns={"MODIFIED_ZCTA": "Zip",
                            "NEIGHBORHOOD_NAME": "Neighborhood",
                            "BOROUGH_GROUP": "Borough",
                            "COVID_CASE_COUNT": "Covid_Case_Count",
                            "COVID_CASE_RATE": "Covid_Case_Rate",
                            "POP_DENOMINATOR": "Pop_Denominator",
                            "COVID_DEATH_COUNT": "Covid_Death_Count",
                            "COVID_DEATH_RATE":"Covid_Death_Rate",
                            "PERCENT_POSITIVE":"Percent_Positive",
                            "TOTAL_COVID_TESTS": "Total_Covid_Tests"})

In [4]:
tests_by_day = tests_by_day.rename(columns={"DATE": "Date",
                             "TOTAL_TESTS": "Total_Tests",
                             "POSITIVE_TESTS": "Positive_Tests",
                             "PERCENT_POSITIVE": "Percent_Positive",
                             "TOTAL_TESTS_7DAYS_AVG": "Total_Tests_7Days_AVG",
                             "POSITIVE_TESTS_7DAYS_AVG": "Positive_Tests_7Days_AVG",
                             "PERCENT_POSITIVE_7DAYS_AVG": "Percent_Positive_7Days_AVG",
                             "INCOMPLETE": "Incomplete"})

In [5]:
tests_by_boro = tests_by_boro.rename(columns={"BOROUGH_GROUP": "Borough",
                             "CASE_RATE": "Case_Rate",
                             "HOSPITALIZED_RATE": "Hospitalized_Rate",
                             "DEATH_RATE": "Death_Rate",
                             "CASE_COUNT": "Case_Count",
                             "HOSPITALIZED_COUNT": "Hospitalized_Count",
                             "DEATH_COUNT": "Death_Count"})

In [6]:
median_income = get_zip_income()
med_income = median_income[['median_household_income', 'lat', 'lng', 'population', 'population_density']]
data = data_by_zip[['Zip', 'Neighborhood','Borough', 'Covid_Case_Count', 'Total_Covid_Tests', 'Covid_Death_Count']]
hm_covid = data.join(med_income)
hm_covid

,Zip,Neighborhood,Borough,Covid_Case_Count,Total_Covid_Tests,Covid_Death_Count,median_household_income,lat,lng,population,population_density
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,479,10193,28,81671,40.750,-73.990,21102,33959.0
1,10002,Chinatown/Lower East Side,Manhattan,1452,24932,160,33218,40.720,-73.990,81410,92573.0
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,701,25829,35,92540,40.730,-73.990,56024,97188.0
3,10004,Financial District,Manhattan,57,1350,1,129313,40.700,-74.020,3089,5519.0
4,10005,Financial District,Manhattan,120,3228,2,124670,40.705,-74.005,7135,97048.0
...,...,...,...,...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,Queens,3136,22996,379,39409,40.600,-73.760,60035,21185.0
173,11692,Arverne/Edgemere,Queens,742,6635,95,43354,40.590,-73.800,18540,18566.0
174,11693,Arverne/Broad Channel,Queens,372,3408,29,50570,40.610,-73.820,11916,11950.0
175,11694,Belle Harbor-Neponsit/Rockaway Park,Queens,833,6892,90,76944,40.580,-73.850,20408,14944.0


In [7]:
tests_by_day = fixing_datetime(tests_by_day)

In [8]:
turnstile2019_df = pd.read_csv('../data/Turnstile_Usage_Data__2019.csv', low_memory=False)
turnstile2020_df = pd.read_csv('../data/Turnstile_Usage_Data__2020.csv', low_memory=False)

In [9]:
turnstile2019_df

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
1,A033,R170,02-00-04,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,6483080,4945335
2,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
3,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389
...,...,...,...,...,...,...,...,...,...,...,...
10467096,PTH02,R544,00-00-03,HARRISON,1,PTH,12/29/2018,23:57:36,REGULAR,114402,15472
10467097,PTH17,R541,01-01-02,THIRTY THIRD ST,1,PTH,12/29/2018,23:58:09,REGULAR,758475,361319
10467098,PTH04,R551,00-00-03,GROVE STREET,1,PTH,12/29/2018,23:58:55,REGULAR,8451,24664
10467099,PTH19,R549,02-02-04,NEWARK C,1,PTH,12/29/2018,23:59:03,REGULAR,26452,2129


In [10]:
ts_df_2019 = fixing_datetime(turnstile2019_df)
ts_df_2020 = fixing_datetime(turnstile2020_df)
ts_df_2019 = clean_up_turnstile_data(ts_df_2019)
ts_df_2020 = clean_up_turnstile_data(ts_df_2020)

../src/support_functions.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True, ascending=False)
../src/support_functions.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True)


In [11]:
gb2019 = ts_df_2019[['Station', 'SCP', 'Date', 'Entries']].groupby(['Date', 'Station', 'SCP']).agg([min, max])
gb2019.reset_index()

gb2019['Total'] = gb2019[('Entries', 'max')] - gb2019[('Entries', 'min')]
gb2019.drop(gb2019[gb2019['Total'] < 10].index, inplace=True)
gb2019.drop(gb2019[gb2019['Total'] > 10000].index, inplace=True)

sum_per_station_2019 = gb2019.groupby(level=[0, 1]).sum()

sum_per_date_2019 = sum_per_station_2019.groupby(level=[0]).sum()

sum_per_date_2019['week_avg_2019'] = sum_per_date_2019.iloc[:,2].rolling(window=7).mean()

In [12]:
gb2020 = ts_df_2020[['Station', 'SCP', 'Date', 'Entries']].groupby(['Date', 'Station', 'SCP']).agg([min, max])
gb2020.reset_index()

gb2020['Total'] = gb2020[('Entries', 'max')] - gb2020[('Entries', 'min')]
gb2020.drop(gb2020[gb2020['Total'] < 10].index, inplace=True)
gb2020.drop(gb2020[gb2020['Total'] > 10000].index, inplace=True)

sum_per_station_2020 = gb2020.groupby(level=[0, 1]).sum()

sum_per_date_2020 = sum_per_station_2020.groupby(level=[0]).sum()

sum_per_date_2020['week_avg_2020'] = sum_per_date_2020.iloc[:,2].rolling(window=7).mean()

## Hypothesis Test on MTA Usage between 2019 and 2020

H0: There was no difference in MTA Turnstile usage between March 01, 2019 thru September 30, 2019 and March 01, 2020 thru September 30, 2020.

HA: There was a difference in MTA Turnstile usage between March 01, 2019 thru September 30, 2019 and March 01, 2020 thru September 30, 2020.

Alpha: 0.05

In [13]:
stats.ttest_ind(sum_per_date_2019['week_avg_2019'].dropna(), sum_per_date_2020['week_avg_2020'].dropna(), equal_var=False )

Ttest_indResult(statistic=58.64502100753585, pvalue=1.126547666215833e-145)

***Conclusion***: Since the p-value is lower than our alpha, we reject the Null Hypothesis 
and can say there is effecient statistical evidence that there was a difference in MTA Turnstile usage between March 01, 2019 thru September 30, 2019 and March 01, 2020 thru September 30, 2020.

## 

In [14]:
sum_per_station_2020

Entries             Total
                                   min         max      
Date       Station                                      
2020-03-01 1 AV                 143577      147805  4228
           103 ST-CORONA      76436221    76445760  9539
           104 ST           1686796086  1686796517   431
           110 ST             40413160    40417505  4345
           111 ST              8060819     8061052   233
...                                ...         ...   ...
2020-09-25 WOODLAWN           32513462    32516320  2858
           WORLD TRADE CTR  1601272180  1601274889  2709
           WTC-CORTLANDT     737036711   737038454  1743
           YORK ST            29532143    29534751  2608
           ZEREGA AV           2617892     2618857   965

[74483 rows x 3 columns]

In [15]:
hm_covid

,Zip,Neighborhood,Borough,Covid_Case_Count,Total_Covid_Tests,Covid_Death_Count,median_household_income,lat,lng,population,population_density
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,479,10193,28,81671,40.750,-73.990,21102,33959.0
1,10002,Chinatown/Lower East Side,Manhattan,1452,24932,160,33218,40.720,-73.990,81410,92573.0
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,701,25829,35,92540,40.730,-73.990,56024,97188.0
3,10004,Financial District,Manhattan,57,1350,1,129313,40.700,-74.020,3089,5519.0
4,10005,Financial District,Manhattan,120,3228,2,124670,40.705,-74.005,7135,97048.0
...,...,...,...,...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,Queens,3136,22996,379,39409,40.600,-73.760,60035,21185.0
173,11692,Arverne/Edgemere,Queens,742,6635,95,43354,40.590,-73.800,18540,18566.0
174,11693,Arverne/Broad Channel,Queens,372,3408,29,50570,40.610,-73.820,11916,11950.0
175,11694,Belle Harbor-Neponsit/Rockaway Park,Queens,833,6892,90,76944,40.580,-73.850,20408,14944.0


In [16]:
subway_station_df = pd.read_csv('../data/Subway_Stations.csv')

In [17]:
subway_station_df

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"
...,...,...,...,...,...,...
468,http://web.mta.info/nyct/service/,469,Coney Island - Stillwell Av,POINT (-73.9812359981396 40.57728100006751),D-F-N-Q,"D,F,N,Q-all times"
469,http://web.mta.info/nyct/service/,470,34th St - Hudson Yards,POINT (-74.00219709442206 40.75544635961596),7-7 Express,"7-all times, 7 Express-rush hours AM westbound..."
470,http://web.mta.info/nyct/service/,641,72nd St,POINT (-73.95836178682246 40.76880251014895),Q,Q-all times
471,http://web.mta.info/nyct/service/,642,86th St,POINT (-73.95177090964917 40.77786104333163),Q,Q-all times


In [18]:
# median_income = get_zip_income()
# med_income = median_income[['median_household_income', 'lat', 'lng', 'population', 'population_density']]
# data = data_by_zip[['Zip', 'Neighborhood','Borough', 'Covid_Case_Count', 'Total_Covid_Tests', 'Covid_Death_Count']]
# hm_covid = data.join(med_income)
# hm_covid